In [1]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize, Bounds

# Define the constant parameters
b_h = 0.0000541  # per capita/birth rate of humans
rho_h = 0.1  # progression rate of humans from exposed to infected class
#alpha_h = 0.15  # rate of production of eggs in the human host
gamma_h = 0.0767  # recovery rate of infected humans
b_a = 0.01096  # per capita/birth rate of animals
rho_a = 0.2  # progression rate of animals from exposed to infected class
#alpha_a = 0.15  # rate of production of eggs in the animal host
gamma_a = 0.01  # recovery rate of infected animals
d_a = 0.000456  # death rate of animals

# Define the model derivative function
def deriv(y, t, b_h, beta_h, rho_h, epsilon_h, gamma_h, b_a, beta_a, rho_a, epsilon_a, gamma_a, d_a, mu_m):
    sh, eh, ih, sa, ea, ia, m = y
    dshdt = b_h + gamma_h * ih - (b_h + beta_h * m / (1 + m)) * sh
    dehdt = (beta_h * m / (1 + m)) * sh - (b_h + rho_h) * eh
    dihdt = rho_h * eh - (b_h + gamma_h) * ih
    dsadt = b_a + gamma_a * ia - ((b_a + beta_a * m / (1 + m)) - d_a  * ia) * sa
    deadt = (beta_a * m / (1 + m)) * sa - (b_a + rho_a - d_a  * ia) * ea
    diadt = rho_a * ea - (b_a + d_a + gamma_a  - d_a  * ia) * ia
    dmdt =  epsilon_h * ih +  epsilon_a * ia - mu_m  * m
    return dshdt, dehdt, dihdt, dsadt, deadt, diadt, dmdt

# Define the STH model function
def STH_model(varied_params, initial_conditions, t, observed_value_humans, observed_value_animals):
    beta_h, epsilon_h, beta_a, epsilon_a, mu_m = varied_params
    result = odeint(deriv, initial_conditions, t, args=(b_h, beta_h, rho_h, epsilon_h, gamma_h, b_a, beta_a, rho_a, epsilon_a, gamma_a, d_a, mu_m))
    sum_of_infected_humans = np.sum(result[-1, 2])
    sum_of_infected_animals = np.sum(result[-1, 5])
    error_humans = np.abs(sum_of_infected_humans - observed_value_humans)
    error_animals = np.abs(sum_of_infected_animals - observed_value_animals)
    #print(sum_of_infected_humans)
    #print(sum_of_infected_animals)
    return error_humans + error_animals

# Set the initial conditions and time points
initial_conditions = [0.8, 0.18, 0.02, 0.7, 0.1, 0.2, 0.3]  # Initial conditions for Sh, Eh, Ih, Sa, Ea, Ia, M
t = np.linspace(0, 365, 1000)  # Time grid for one year

# Observed values you are interested in
observed_value_of_interest_humans = 0.196  # Observed prevalence of STH in humans
observed_value_of_interest_animals = 0.56  # Observed prevalence of STH in animals

# Define the parameter space to explore with bounds
#beta_h_bounds = [0.1, 0.9]
#beta_a_bounds = [0.1, 0.9]
beta_h_bounds = [0.0, 0.1]
epsilon_h_bounds = [0.01, 0.2]
beta_a_bounds = [0.01, 0.2]
epsilon_a_bounds = [0.01, 0.2]
mu_m_bounds = [0.00035, 0.5]

# Initialize variables for optimal parameters and corresponding error
optimal_varied_params = None
min_error = float('inf')

L=1000
beta_vec = np.linspace(beta_h_bounds[0],beta_h_bounds[1],L)
epsilon_vec = np.linspace(epsilon_h_bounds[0],epsilon_h_bounds[1],L)
beta_a_vec = np.linspace(beta_a_bounds[0],beta_a_bounds[1],L)
epsilon_a_vec = np.linspace(epsilon_a_bounds[0],epsilon_a_bounds[1],L)
mu_vec = np.linspace(mu_m_bounds[0],mu_m_bounds[1],L)


for i in range(5000):
    rand_vec = np.random.choice(L,size=5)
    varied_params = [beta_vec[rand_vec[0]], epsilon_vec[rand_vec[1]], beta_a_vec[rand_vec[2]], epsilon_a_vec[rand_vec[3]], mu_vec[rand_vec[4]]]
    err = STH_model(varied_params, initial_conditions, t, observed_value_of_interest_humans, observed_value_of_interest_animals)
    if err <= min_error:
        min_error = err
        optimal_varied_params = varied_params
    if i%100==0:
        print(f'Loop {i}: {optimal_varied_params}, Error: {min_error}')
    i += 1

# Extract and print the optimal parameters
print("Optimal Parameters (beta_h, epsilon_h, beta_a, epsilon_a, mu_m):", optimal_varied_params,"Error: %0.3f" %(min_error))

# Simulate the STH model with optimal parameters
solution = odeint(deriv, initial_conditions, t, args=(b_h, optimal_varied_params[0], rho_h, optimal_varied_params[1], gamma_h, b_a, optimal_varied_params[2], rho_a, optimal_varied_params[3], gamma_a, d_a, optimal_varied_params[4]))


Loop 0: [0.05655655655655656, 0.03852852852852853, 0.010570570570570571, 0.04936936936936937, 0.2039111111111111], Error: 0.7553585116848733
Loop 100: [0.08888888888888889, 0.12962962962962962, 0.09311311311311311, 0.16975975975975977, 0.2379213213213213], Error: 0.04287487502077397
Loop 200: [0.08888888888888889, 0.12962962962962962, 0.09311311311311311, 0.16975975975975977, 0.2379213213213213], Error: 0.04287487502077397
Loop 300: [0.07607607607607608, 0.023313313313313315, 0.09634634634634634, 0.16652652652652655, 0.19940975975975975], Error: 0.01881185926573739
Loop 400: [0.07607607607607608, 0.023313313313313315, 0.09634634634634634, 0.16652652652652655, 0.19940975975975975], Error: 0.01881185926573739
Loop 500: [0.07607607607607608, 0.023313313313313315, 0.09634634634634634, 0.16652652652652655, 0.19940975975975975], Error: 0.01881185926573739
Loop 600: [0.08648648648648649, 0.05754754754754755, 0.12544544544544545, 0.1341941941941942, 0.2524256756756757], Error: 0.01520001506518